In [1]:
# Gerekli kütüphaneler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

# Haritalar ve jeotermal veri görselleştirme için
import folium


C:\Users\tolgakilinckaya\AppData\Local\Temp\ipykernel_16220\3364035857.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Deprem verilerini yükleyin
deprem_veri = pd.read_csv('turkey_earthquakes(1915-2024_feb).csv')

# Jeotermal kaynak verilerini yükleyin (örneğin bir CSV dosyası)
jeotermal_veri = pd.read_csv('temperature_data.csv')


In [3]:
# Deprem verileri
print(deprem_veri.head())

# Jeotermal veriler
print(jeotermal_veri.head())

# Gerekli sütunlar
deprem_veri = deprem_veri[['Enlem', 'Boylam', 'Derinlik', 'xM', 'Olus tarihi']]
jeotermal_veri = jeotermal_veri[['Latitude', 'Longitude', 'Temperature']]


   No     Deprem Kodu Olus tarihi  Olus zamani    Enlem   Boylam  Derinlik  \
0   1  20231231222846  2023.12.31  22:28:46.51  37.4900  43.7095       2.6   
1   2  20231231170605  2023.12.31  17:06:05.42  37.4405  43.9263       5.0   
2   3  20231231165304  2023.12.31  16:53:04.13  37.4942  43.6955       5.0   
3   4  20231231163755  2023.12.31  16:37:55.84  37.4763  43.6965       5.0   
4   5  20231230204324  2023.12.30  20:43:24.21  40.2272  29.1705      12.9   

    xM   MD   ML   Mw   Ms   Mb Tip  \
0  3.8  0.0  3.8  3.8  0.0  0.0  Ke   
1  4.5  0.0  4.5  4.4  0.0  0.0  Ke   
2  3.5  0.0  3.5  3.4  0.0  0.0  Ke   
3  4.6  0.0  4.6  4.4  0.0  0.0  Ke   
4  3.6  0.0  3.6  3.4  0.0  0.0  Ke   

                                               Yer  
0                 BAYKOY- (HAKKARI) [South 6.8 km]  
1  YAZILI-YUKSEKOVA (HAKKARI) [South West  3.6 km]  
2                 CIMENLI- (HAKKARI) [East 6.1 km]  
3          CIMENLI- (HAKKARI) [South East  6.3 km]  
4               GURSU (BURSA) [

In [4]:
# Türkiye haritası
turkiye_harita = folium.Map(location=[39.0, 35.0], zoom_start=6)

# RGB'den hex'e dönüştürme fonksiyonu
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])

# Sıcaklığa göre renk kodlarını belirleyin
def get_color(temperature):
    if temperature >= 85:
        return rgb_to_hex([153, 0, 0])  # Koyu kırmızı
    elif temperature >= 55:
        return rgb_to_hex([255, 0, 0])  # Kırmızı
    elif temperature >= 40:
        return rgb_to_hex([255, 102, 0])  # Koyu turuncu
    elif temperature >= 33:
        return rgb_to_hex([255, 204, 102])  # Turuncu
    elif temperature >= 29:
        return rgb_to_hex([255, 255, 102])  # Açık sarı
    elif temperature >= 28:
        return rgb_to_hex([204, 255, 255])  # Maviye yakın beyaz
    elif temperature >= 27:
        return rgb_to_hex([153, 204, 255])  # Çok açık mavi
    elif temperature >= 25:
        return rgb_to_hex([102, 153, 255])  # Daha açık mavi
    elif temperature >= 22:
        return rgb_to_hex([51, 102, 255])  # Açık mavi
    else:
        return rgb_to_hex([0, 0, 255])  # Mavi

# Türkiye haritasını oluşturun
turkiye_harita = folium.Map(location=[39.0, 35.0], zoom_start=6)

# Jeotermal kaynakları haritaya ekleyin
for idx, row in jeotermal_veri.iterrows():
    color = get_color(row['Temperature'])
    folium.CircleMarker(location=(row['Latitude'], row['Longitude']),
                        radius=5,
                        color=color,
                        fill=True,
                        fill_color=color,
                        popup=f'Sıcaklık: {row["Temperature"]} °C').add_to(turkiye_harita)

        
# Haritayı kaydet
turkiye_harita.save('jeotermal_sicaklik_haritasi.html')

In [6]:
def get_color_earthquakes(xM):
    if xM >= 6.5:
        return rgb_to_hex([127, 255, 0])  # Açık yeşil
    elif xM >= 4.5:
        return rgb_to_hex([0, 191, 0])  # Orta yeşil
    else:
        return rgb_to_hex([0, 127, 0])  # Koyu yeşil

# Depremler
for idx, row in deprem_veri.iterrows():
    color = get_color_earthquakes(row['xM'])
    folium.CircleMarker(location=(row['Enlem'], row['Boylam']),
                            radius=3,
                            color=color,
                            fill=True,
                            fill_color=color,
                            popup=f'Büyüklük: {row["xM"]}').add_to(turkiye_harita)


# Haritayı kaydet
turkiye_harita.save('jeotermal_ve_deprem_haritasi.html')

# Korelasyon İnceleme

In [7]:
from scipy.spatial import cKDTree

# Deprem ve jeotermal noktaların koordinatlarını alalım
deprem_coords = deprem_veri[['Enlem', 'Boylam']].values
jeotermal_coords = jeotermal_veri[['Latitude', 'Longitude']].values

# KDTree kullanarak en yakın jeotermal kaynakları bulalım
tree = cKDTree(jeotermal_coords)
distances, indices = tree.query(deprem_coords, k=1)

# En yakın jeotermal kaynakların sıcaklıklarını alalım
nearest_temps = jeotermal_veri.iloc[indices]['Temperature'].values

# Deprem verilerine bu sıcaklıkları ekleyelim
deprem_veri['nearest_temp'] = nearest_temps

# Korelasyon hesaplayalım
correlation = deprem_veri[['xM', 'nearest_temp']].corr()
print("Deprem büyüklüğü ile en yakın jeotermal sıcaklık arasındaki korelasyon:")
print(correlation)


Deprem büyüklüğü ile en yakın jeotermal sıcaklık arasındaki korelasyon:
                   xM  nearest_temp
xM            1.00000       0.01718
nearest_temp  0.01718       1.00000
